### Task 4.2 - Visualization - Interactive Dashboard

In [1]:
import pandas as pd
from dash import dash, html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
from plotly.subplots import make_subplots

In [2]:
# load data
df = pd.read_csv("datasets/falcon9_dataset.csv")

In [3]:
# prepare lists for dashboard selectors
orbits = [orbit for orbit in df["Orbit"].unique()]
orbits = sorted(orbits)
orbits.insert(0, "All Orbits")

blocks = [str(block) for block in df["Block"].unique()]
blocks.insert(0, "All Blocks")

In [4]:
# Dashboard
app = dash.Dash(__name__)

app.layout = html.Div(
    children = [
        html.Center(html.H2("Flacon 9 Launches Dashboard", style={'color': '#2B2B2B', 'fontsize':'35', 'font-family':'Verdana'})),

        # Bar chart - number of flights for specified year and site
        dcc.Graph(id='bar-chart'),
        
        # Drop down - sites
        html.Center(
            dcc.Dropdown(
            id="drop-down",
            options=['All Sites', 'CCSFS SLC 40', 'VSFB SLC 4E', 'KSC LC 39A'],
            value='All Sites',
            style={'width': '250px', 'textAlign': 'center'},
            clearable=False
            )
        ),
        html.Br(),

        # Range slider - year
        dcc.RangeSlider(
        id='year-slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=[2010, 2023],
        step=1,
        marks={str(year): str(year) for year in df['Year'].unique()}
        ),

        # Pie charts - success/fail rate for each site
        dcc.Graph(id='pie-chart'),

        html.Hr(),
        html.Br(),
        html.Br(),
        
        # Scatter plot - success/fail outcome for specified orbit and payload mass
        dcc.Graph(id='scatter-plot'),

        # Radio items - orbit
        html.Center(
            dcc.RadioItems(
                id="orbit-item",
                options=orbits,
                value='All Orbits',
                inline=True,
                style={'font-family': 'sans-serif'}
            )
        ),

        html.Br(),

        # Range slider - payload mass
        dcc.RangeSlider(
            id='mass-slider',
            min=df['PayloadMass'].min(),
            max=df['PayloadMass'].max(),
            value=(0, df['PayloadMass'].max()),
            tooltip={"placement": "bottom", "always_visible": True}
        ),
        html.Center(html.H5("payload mass range", style={'color': '#3B3B3B', 'fontsize':'3', 'font-family':'sans-serif'}))

    ]
)



@app.callback(
        Output('bar-chart', 'figure'),
        [Input('year-slider', 'value'),
        Input('drop-down', 'value')]
        )
def update_bar_chart(year_value, site_value):
    """
    Input: year, site
    Outpot: figure
    """
    min, max= year_value
    years = [year for year in range(int(min), int(max)+1)]

    data = df[df["Year"].isin(years)].copy()
    data["Outcome"] = data['Outcome'].replace({1: 'Success', 0: 'Fail'})

    # bar chart for all sites
    if site_value == 'All Sites':
        data = data.groupby(["Year", "Outcome"]).size().reset_index()
        data = data.rename(columns={0:'Flights'})
        
        years_order = [str(year) for year in data['Year'].unique()]
        data['Year'] = data['Year'].astype(str)
    
        bar_fig = px.bar(data, x='Year', y='Flights',
                         labels={'Flights':'Number of Flights'}, color='Outcome',
                         color_discrete_map={'Success': 'skyblue', 'Fail': 'lightcoral'},
                         category_orders={'Year':years_order, 'Outcome': ['Success', 'Fail']},
                         custom_data=['Outcome'])
    
    else:
        # bar chart for specified site
        data = data[data['LaunchSite'] == site_value]
        data = data.groupby(["Year", "Outcome"]).size().reset_index()
        data = data.rename(columns={0:'Flights'})

        years_order = [str(year) for year in data['Year'].unique()]
        data['Year'] = data['Year'].astype(str)

        bar_fig = px.bar(data, x='Year', y='Flights',
                         labels={'Flights':'Number of Flights'}, color='Outcome',
                         color_discrete_map={'Success': 'skyblue', 'Fail': 'lightcoral'},
                         category_orders={'Year':years_order, 'Outcome': ['Success', 'Fail']},
                         custom_data=['Outcome'])
    
    bar_fig.update_layout(
        title_text="Number of flights for each year",
        title_font={'family': 'Arial',
                    'color': '#182A40',
                    'size': 22})

    bar_fig.update_traces(
        hovertemplate="<br>".join([
            '<b>%{customdata[0]}</b>',
            'Year: %{x}',
            'Number of flight: %{y}'
        ]))
    return bar_fig


@app.callback(
        Output('pie-chart', 'figure'),
        Input('year-slider', 'value')
        )
def update_pie_chart(year_value):
    min, max = year_value
    years = [year for year in range(int(min), int(max)+1)]

    pie_fig = make_subplots(rows = 1, cols = 4,
                        specs=[[{'type':'pie'},{'type':'pie'},{'type':'pie'}, {'type':'pie'}]],
                        subplot_titles=('Total', 'CCSFS SLC 40', 'VSFB SLC 4E', 'KSC LC 39A'))
    
    # Total success/fail rate pie chart
    total = df[df["Year"].isin(years)].copy()
    total["Outcome"] = total['Outcome'].replace({1: 'Success', 0: 'Fail'})
    total_values = total.groupby(["Outcome"]).size()
    total_group = total_values.index
    total_pie = px.pie(values=total_values.array, names=total_group,
                       color=total_group, color_discrete_map={'Success': 'skyblue', 'Fail': 'lightcoral'})

    for trace in total_pie.data:
        pie_fig.add_trace(trace, row=1, col=1)

    # Site 1 - CCSFS SLC 40 success/fail rate pie chart
    CSSFS = total[total["LaunchSite"] == "CCSFS SLC 40"]
    CSSFS_values = CSSFS.groupby(["Outcome"]).size()
    CSSFS_group = CSSFS_values.index
    CSSFS_pie = px.pie(values=CSSFS_values.array, names=CSSFS_group,
                       color=CSSFS_group, color_discrete_map={'Success': 'skyblue', 'Fail': 'lightcoral'})

    for trace in CSSFS_pie.data:
        pie_fig.add_trace(trace, row=1, col=2)

    # Site 2 - VSFB SLC 4E success/fail rate pie chart
    VSFB = total[total["LaunchSite"] == "VSFB SLC 4E"]
    VSFB_values = VSFB.groupby(["Outcome"]).size()
    VSFB_group = VSFB_values.index
    VSFB_pie = px.pie(values=VSFB_values.array, names=VSFB_group,
                      color=VSFB_group, color_discrete_map={'Success': 'skyblue', 'Fail': 'lightcoral'})

    for trace in VSFB_pie.data:
        pie_fig.add_trace(trace, row=1, col=3)

    # Site 3 - KSC LC 39A success/fail rate pie chart
    KSC = total[total["LaunchSite"] == "KSC LC 39A"]
    KSC_values = KSC.groupby(["Outcome"]).size()
    KSC_group = KSC_values.index
    KSC_pie = px.pie(values=KSC_values.array, names=KSC_group,
                     color=KSC_group, color_discrete_map={'Success': 'skyblue', 'Fail': 'lightcoral'})

    for trace in KSC_pie.data:
        pie_fig.add_trace(trace, row=1, col=4)
    
    pie_fig.update_layout(
        title_text="Success/Fail rate for each site",
        title_font={'family': 'Arial',
                    'color': '#182A40',
                    'size': 22})
    
    pie_fig.update_traces(
        hovertemplate="<br>".join([
            '<b>%{label}</b>',
            'Number of flight: %{value}'
        ]))
    return pie_fig

@app.callback(
        Output('scatter-plot', 'figure'),
        [Input('orbit-item', 'value'),
         Input('mass-slider', 'value')]
        )
def update_scatter(orbit_value, mass_value):
    min, max = mass_value
    
    data = df.copy()
    # filter the payload mass range
    data = data[(data["PayloadMass"] >= min) & (data["PayloadMass"] <= max)]
    data["Outcome"] = data['Outcome'].replace({1: 'Success', 0: 'Fail'})
    data['Block'] = data['Block'].astype(str)

    color_mapping = {
        '1.0': '#636EFA',
        '2.0': '#EF553B',
        '3.0': '#00CC96',
        '4.0': '#AB63FA',
        '5.0': '#FFA15A'
    }

    symbol_mapping = {
        '1.0': 'circle',
        '2.0': 'diamond',
        '3.0': 'square',
        '4.0': 'x',
        '5.0': 'cross'
    }

    # scatter plot for all orbits
    if orbit_value == 'All Orbits':
        scatter_fig = px.scatter(data, x='PayloadMass' ,y='Outcome',
                                 color='Block', color_discrete_map=color_mapping,
                                 symbol='Block',symbol_map=symbol_mapping,
                                 range_y=["Fail", 'Success'],
                                 custom_data=['FlightNumber', 'Date', 'LaunchSite', 'Block'])
    else:
        # scatter plot for specified orbit
        data = data[data['Orbit'] == orbit_value]
        scatter_fig = px.scatter(data, x='PayloadMass', y='Outcome',
                                 color='Block', color_discrete_map=color_mapping,
                                 symbol='Block', symbol_map=symbol_mapping,
                                 range_y=["Fail", 'Success'],
                                 custom_data=['FlightNumber', 'Date', 'LaunchSite', 'Block'])
    
    scatter_fig.update_layout(
        title_text="Outcome of each Orbit and Block",
        title_font={'family': 'Arial',
                    'color': '#182A40',
                    'size': 22})
    
    scatter_fig.update_traces(
        hovertemplate="<br>".join([
            '<b>Flight number: %{customdata[0]}</b>',
            'Date: %{customdata[1]}',
            'Launch site: %{customdata[2]}',
            'Block: %{customdata[3]}',
            'Payload mass: %{x} kg',
            'Outcome: %{y}'
        ]))
    
    scatter_fig.update_xaxes(title='Payload Mass (kg)')
    
    return scatter_fig

if __name__ == '__main__':
    app.run_server(debug=True)